In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import itertools
from tqdm import tqdm
import seaborn as sns
from folk_tables_utils import *


In [ ]:
plt.rc('xtick', labelsize=12)   
plt.rc('ytick', labelsize=12)   
plt.rc('legend', fontsize=12)
plt.rc('font', family='serif', serif='Palatino')

# Make Dataset objects

### Gaussian

In [ ]:
def generate_well_conditioned_covariance_matrix(dim, condition_number=10):
    # Generate a random orthogonal matrix using QR decomposition
    random_matrix = np.random.randn(dim, dim)
    Q, _ = np.linalg.qr(random_matrix)

    # Generate random eigenvalues with the desired condition number
    min_eigenvalue = 1
    max_eigenvalue = min_eigenvalue * condition_number
    eigenvalues = np.random.uniform(min_eigenvalue, max_eigenvalue, dim)

    # Create a diagonal matrix with the generated eigenvalues
    diag_eigenvalues = np.diag(eigenvalues)

    # Construct the covariance matrix using the orthogonal matrix and diagonal eigenvalue matrix
    covariance_matrix = Q @ diag_eigenvalues @ Q.T

    return covariance_matrix

def generate_spiked_covariance(n, k, scale=10):
    """
    Generates a full rank covariance matrix with k eigenvalues larger than the rest.

    Args:
    n (int): The dimensionality of the covariance matrix.
    k (int): The number of large eigenvalues.

    Returns:
    np.ndarray: The generated covariance matrix.
    """
    assert k <= n, "k should be less than or equal to n"

    # Generate n random eigenvalues
    eigenvalues = np.random.rand(n)

    # Increase the first k eigenvalues
    eigenvalues[:k] *= scale

    # Generate a random orthogonal matrix
    Q, _ = np.linalg.qr(np.random.randn(n, n))

    # Generate the covariance matrix
    covariance = Q @ np.diag(eigenvalues) @ Q.T

    return covariance



In [ ]:
np.random.seed(0)
SubmodelClass = LinearRegressionModel
ImputedModelClass = ImputedLinearRegressionModel
AggregatorClass = MyModelAggregator

# Gaussian Dataset
all_gaus_cols = np.arange(30)

# generate a list of random subsets of all_gaus_cols
dataset_gaus_cols_list = [np.random.choice(all_gaus_cols, 20, replace=False) for i in range(10)]
dataset_gaus_cols_list += [np.random.choice(all_gaus_cols, 15, replace=False) for i in range(20)]
num_datasets = len(dataset_gaus_cols_list)

# generate a random symmetric covariance matrix
A = np.random.normal(0, 1, (len(all_gaus_cols), 100))
cov_matrix = A @ A.T / 100
cov_matrix = generate_well_conditioned_covariance_matrix(len(all_gaus_cols), condition_number=100)
cov_matrix = generate_spiked_covariance(len(all_gaus_cols), 3, scale=100)

theta = np.random.normal(0, 50, len(all_gaus_cols))
theta = theta[np.argsort(np.abs(theta))][::-1]
sigma = 1.0
# assert cov_matrix is symmetric
assert np.allclose(cov_matrix, cov_matrix.T)
sns.heatmap(np.abs(cov_matrix))
#label the axes and legend
plt.xlabel('Feature Index', fontsize=20)
plt.ylabel('Feature Index', fontsize=20)
plt.title('Covariance Matrix Heatmap', fontsize=20)
plt.tight_layout()
# plt.savefig("gaussian-plots/gaussian-covariance-matrix-heatmap.pdf")
theta

In [ ]:
ub_lb_dataset = GaussianDataset(theta=theta, cov=cov_matrix, sigma=sigma, columns=all_gaus_cols, seed=0)
og_dataset_list = [GaussianDataset(theta=theta, cov=cov_matrix, sigma=sigma, columns=all_gaus_cols, seed=0) for i in range(num_datasets)]
dataset_list = [GaussianDataset(theta=theta, cov=cov_matrix, sigma=sigma, columns=dataset_gaus_cols_list[i], seed=0) for i in range(num_datasets)]
og_val_dataset_list = og_dataset_list
val_dataset_list = dataset_list
og_test_dataset_list = og_dataset_list
test_dataset_list = dataset_list

all_og_dataset = og_dataset_list[-1]

# Train Base Models

In [ ]:
def make_gaussian_imse_fn(gds):
    assert np.array_equal(all_gaus_cols, np.arange(len(gds.theta)))
    perm_plus, perm_minus = make_projection_permutation(all_gaus_cols, gds.columns)
    perm = perm_plus + perm_minus
    cov_mat = gds.cov
    Sig_iplus = cov_mat[perm_plus, :][:, perm_plus]
    Sig_iplus_inv = np.linalg.inv(Sig_iplus)
    Sig_ipm = cov_mat[perm_plus, :][:, perm_minus]
    A = Sig_iplus_inv @ Sig_ipm
    def gaussian_imse_fn(pred, target):
        pred_plus = pred[perm_plus]
        pred_minus = pred[perm_minus]

        target_plus = target[perm_plus]
        target_minus = target[perm_minus]

        new_pred = pred_plus + A @ pred_minus
        new_target = target_plus + A @ target_minus
        v = new_pred - new_target
        return v.T @ Sig_iplus @ v / len(v)
    return gaussian_imse_fn

itds = dataset_list[-1]


gaussian_imse_fn = make_gaussian_imse_fn(itds)
# gaussian_imse_fn = lambda x, y: -1.0

In [ ]:
n_list = [50, 100, 200, 500, 1000]
# n_list = [2000, 3000, 4000, 5000, 6000, 7000, 8000, 10000]
trial_list = np.arange(20)
results_list = []
result_cols = ['n', 'dataset_idx', 'num_columns', 'mse', 'abse', 'bool_error', 'trial', 'gaussian_imse']
models_list = [SubmodelClass() for _ in range(num_datasets)]
for n, trial in tqdm(itertools.product(n_list, trial_list)):
    tds = og_test_dataset_list[-1]
    cov_matrix, _, _ = compute_covariance_matrix(og_dataset_list[-1])
    imputed_baseline = ImputedModelClass(cov_matrix, og_dataset_list[-1].columns)
    rw_imputed_baseline = ImputedModelClass(cov_matrix, og_dataset_list[-1].columns)
    
    losses_list = []
    for i in range(num_datasets):
        ds = dataset_list[i]
        ds.shuffle(seed=trial)

        X, y, columns = ds.get_n_samples_numpy(n)
        imputed_baseline.impute(X, y, columns)
        models_list[i].fit(X, y, columns)

        vds = val_dataset_list[i]
        mse = compute_error(vds, model=models_list[i], metric=mse_fn, num_samples=10000, imperfect=True)
        losses_list.append(mse)
        rw_imputed_baseline.impute(X, y, columns, reweight=1 / mse)

        gaussian_imse = compute_error(tds, model=models_list[i], metric=gaussian_imse_fn, num_samples=6000, imperfect=False)
        mse = compute_error(tds, model=models_list[i], metric=hacky_cov_mse_fn, num_samples=6000, imperfect=False)
        abse = compute_error(tds, model=models_list[i], metric=abse_fn, num_samples=6000, imperfect=False)
        bool_error = compute_error(tds, model=models_list[i], metric=boolerr_fn, num_samples=6000, imperfect=False)
        results_list.append((n, str(i), str(len(columns)), mse, abse, bool_error, trial, gaussian_imse))

    agg_model = AggregatorClass()
    losses_list = np.array(losses_list)
    agg_model.fit(all_columns=og_dataset_list[-1].columns, model_list=models_list, cov_matrix=cov_matrix, losses_list=losses_list)
    gaussian_imse = compute_error(tds, model=agg_model, metric=gaussian_imse_fn, num_samples=6000, imperfect=False)
    mse = compute_error(tds, model=agg_model, metric=hacky_cov_mse_fn, num_samples=6000, imperfect=False)
    abse = compute_error(tds, model=agg_model, metric=abse_fn, num_samples=6000, imperfect=False)
    bool_error = compute_error(tds, model=agg_model, metric=boolerr_fn, num_samples=6000, imperfect=False)
    results_list.append((n, "agg", "NA", mse, abse, bool_error, trial, gaussian_imse))

    myds = og_dataset_list[-1]
    myds.shuffle(seed=trial)
    myfeatures, mylabels, mycolumns = myds.get_n_samples_numpy(n)

    identity_cov_ds = GaussianDataset(theta=tds.theta, cov=np.eye(tds.theta.shape[0]), sigma=0.0, columns=tds.columns)
    naive_agg_model = NaiveAggregator(all_columns=og_dataset_list[-1].columns, model_list=models_list)
    gaussian_imse = -1
    mse = compute_error(tds, model=naive_agg_model, metric=mse_fn, num_samples=6000, imperfect=True)
    #parameter error option
    # mse = compute_error(identity_cov_ds, model=naive_agg_model, metric=mse_fn, num_samples=6000, imperfect=True)
    abse = compute_error(identity_cov_ds, model=naive_agg_model, metric=abse_fn, num_samples=6000, imperfect=True)
    bool_error = compute_error(identity_cov_ds, model=naive_agg_model, metric=boolerr_fn, num_samples=6000, imperfect=True)
    results_list.append((n, "naive_agg", "NA", mse, abse, bool_error, trial, gaussian_imse))

    naive_agg_model.fit(myfeatures, mylabels, X_columns=mycolumns)
    gaussian_imse = -1
    mse = compute_error(tds, model=naive_agg_model, metric=mse_fn, num_samples=6000, imperfect=True)
    #parameter error option
    # mse = compute_error(identity_cov_ds, model=naive_agg_model, metric=mse_fn, num_samples=6000, imperfect=True)
    abse = compute_error(identity_cov_ds, model=naive_agg_model, metric=abse_fn, num_samples=6000, imperfect=True)
    bool_error = compute_error(identity_cov_ds, model=naive_agg_model, metric=boolerr_fn, num_samples=6000, imperfect=True)
    results_list.append((n, "opt_naive_agg", "NA", mse, abse, bool_error, trial, gaussian_imse))

    imputed_baseline.fit()
    gaussian_imse = compute_error(tds, model=imputed_baseline, metric=gaussian_imse_fn, num_samples=6000, imperfect=False)
    mse = compute_error(tds, model=imputed_baseline, metric=hacky_cov_mse_fn, num_samples=6000, imperfect=False)
    abse = compute_error(tds, model=imputed_baseline, metric=abse_fn, num_samples=6000, imperfect=False)
    bool_error = compute_error(tds, model=imputed_baseline, metric=boolerr_fn, num_samples=6000, imperfect=False)
    results_list.append((n, "imputed_baseline", "NA", mse, abse, bool_error, trial, gaussian_imse))

    rw_imputed_baseline.fit()
    gaussian_imse = compute_error(tds, model=rw_imputed_baseline, metric=gaussian_imse_fn, num_samples=6000, imperfect=False)
    mse = compute_error(tds, model=rw_imputed_baseline, metric=hacky_cov_mse_fn, num_samples=6000, imperfect=False)
    abse = compute_error(tds, model=rw_imputed_baseline, metric=abse_fn, num_samples=6000, imperfect=False)
    bool_error = compute_error(tds, model=rw_imputed_baseline, metric=boolerr_fn, num_samples=6000, imperfect=False)
    results_list.append((n, "rw_imputed_baseline", "NA", mse, abse, bool_error, trial, gaussian_imse))

    # HACK to get the right number of columns ######
    myds = itds
    # myds = dataset_list[0]
    # myds = ub_lb_dataset
    myds.shuffle(seed=trial)
    myfeatures, mylabels, mycolumns = myds.get_n_samples_numpy(n)
    # ######

    baseline_model = SubmodelClass()
    baseline_model.fit(myfeatures, mylabels, mycolumns)
    gaussian_imse = compute_error(tds, model=baseline_model, metric=gaussian_imse_fn, num_samples=6000, imperfect=False)
    mse = compute_error(tds, model=baseline_model, metric=hacky_cov_mse_fn, num_samples=6000, imperfect=False)
    abse = compute_error(tds, model=baseline_model, metric=abse_fn, num_samples=6000, imperfect=False)
    bool_error = compute_error(tds, model=baseline_model, metric=boolerr_fn, num_samples=6000, imperfect=False)
    results_list.append((n, "lb_baseline", "NA", mse, abse, bool_error, trial, gaussian_imse))

    myfeatures, mylabels, mycolumns = myds.get_n_samples_numpy(num_datasets * n)
    baseline_model = SubmodelClass()
    baseline_model.fit(myfeatures, mylabels, mycolumns)
    gaussian_imse = compute_error(tds, model=baseline_model, metric=gaussian_imse_fn, num_samples=6000, imperfect=False)
    mse = compute_error(tds, model=baseline_model, metric=hacky_cov_mse_fn, num_samples=6000, imperfect=False)
    abse = compute_error(tds, model=baseline_model, metric=abse_fn, num_samples=6000, imperfect=False)
    bool_error = compute_error(tds, model=baseline_model, metric=boolerr_fn, num_samples=6000, imperfect=False)
    results_list.append((n, "ub_baseline", "NA", mse, abse, bool_error, trial, gaussian_imse))


In [ ]:
results_df = pd.DataFrame(results_list, columns=result_cols)
og_results_df = results_df.copy()

In [ ]:
results_df = og_results_df.copy()
method_name_remap = {
    "agg": "Collab",
    "rw_imputed_baseline": "RW-Imputation",
    "imputed_baseline": "Imputation",
    "naive_agg": "Naive-Collab",
    "opt_naive_agg": "Optimized-Naive-Collab",
    "lb_baseline": "Naive-Local",
    "ub_baseline": f"Naive-Local ({num_datasets}x data)"
}
results_df = results_df[results_df["dataset_idx"].isin(method_name_remap.keys())]
results_df['dataset_idx'] = results_df['dataset_idx'].map(method_name_remap)

unique_dataset_idx = results_df['dataset_idx'].unique()

# Create a color palette and a marker list
palette = dict(zip(unique_dataset_idx, sns.color_palette(n_colors=len(unique_dataset_idx))))
markers = dict(zip(unique_dataset_idx, ['o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X'][:len(unique_dataset_idx)]))


### Gaussian

In [ ]:
plot_methods_list = ["Collab", "RW-Imputation", "Imputation", 'Naive-Local']#, "ub_baseline"]
plotdf = results_df[results_df['dataset_idx'].isin(plot_methods_list)]

# log scale mse column in plotdf
plotdf['gaussian_imse'] = np.log10(plotdf['gaussian_imse'])
ax = sns.lineplot(x="n", y="gaussian_imse", hue="dataset_idx", data=plotdf, style="dataset_idx", markersize=7, palette=palette, markers=markers)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles[0:], labels=labels[0:])

plt.ylabel("Mean Sq. Pred. Err. (log)", fontsize=20)
plt.xlabel("number of samples (n)", fontsize=20)
plt.title("Local Feature Prediction Error", fontsize=20)
plt.tight_layout()
# plt.savefig("gaussian-plots/gaussian-local-prediction.pdf")

In [ ]:
plot_methods_list = ["Collab", "RW-Imputation", "Imputation", "Naive-Collab", "Optimized-Naive-Collab"]
plotdf = results_df[results_df['dataset_idx'].isin(plot_methods_list)]

# log scale mse column in plotdf
plotdf['mse'] = np.log10(plotdf['mse'])
ax=sns.lineplot(x="n", y="mse", hue="dataset_idx", data=plotdf, style="dataset_idx", markersize=7, palette=palette, markers=markers)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles[0:], labels=labels[0:])
plt.ylabel("Mean Sq. Pred. Err.", fontsize=20)
plt.xlabel("number of samples (n)", fontsize=20)
plt.title("Full Feature Prediction Error", fontsize=20)
plt.tight_layout()

# plt.savefig("gaussian-plots/gaussian-full-prediction.pdf")

